# Kvikio demo

To get this to work we need
1. https://github.com/zarr-developers/zarr-python/pull/934
2. https://github.com/pydata/xarray/pull/6874

In [1]:
%load_ext watermark

# These imports are currently unnecessary. I import them to show versions
# cupy_xarray registers the kvikio entrypoint on install.
import cupy as cp
#import cudf
import cupy_xarray
import kvikio.zarr

import flox
import numpy_groupies
import numpy as np
import xarray as xr
import zarr

store = "./air-temperature.zarr"

%watermark -iv

flox          : 0.5.10.dev5+g44f3851.d20220816
cupy          : 11.0.0
json          : 2.0.9
cupy_xarray   : 0.1.0+11.gaa2dc91.dirty
numpy         : 1.22.4
zarr          : 2.12.1.dev68
numpy_groupies: 0.9.19+1.g8f14bbf
kvikio        : 22.10.0a0+22.gd063a3b
xarray        : 2022.6.1.dev51+g5a9a51ba1
sys           : 3.8.13 | packaged by conda-forge | (default, Mar 25 2022, 06:04:18) 
[GCC 10.3.0]



## Create example dataset

- cannot be compressed

In [11]:
airt = xr.tutorial.open_dataset("air_temperature", engine="netcdf4")
for var in airt.variables:
    airt[var].encoding["compressor"] = None
airt.to_zarr(store, mode="w", consolidated=True)

/glade/u/home/dcherian/python/xarray/xarray/core/dataset.py:2066: SerializationWarning: saving variable None with floating point data as an integer dtype without any _FillValue to use for NaNs
  return to_zarr(  # type: ignore


## Test opening

### Standard usage

In [4]:
xr.open_dataset(store, engine="zarr").air

<xarray.DataArray 'air' (time: 2920, lat: 25, lon: 53)>
[3869000 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Attributes:
    GRIB_id:       11
    GRIB_name:     TMP
    actual_range:  [185.16000366210938, 322.1000061035156]
    dataset:       NMC Reanalysis
    level_desc:    Surface
    long_name:     4xDaily Air temperature at sigma level 995
    parent_stat:   Other
    precision:     2
    statistic:     Individual Obs
    units:         degK
    var_desc:      Air temperature

### Now with kvikio!

 - must read with `consolidated=False` (https://github.com/rapidsai/kvikio/issues/119)
 - dask.from_zarr to GDSStore / open_mfdataset

In [ ]:
# Consolidated must be False
ds = xr.open_dataset(store, engine="kvikio", consolidated=False)
print(ds.air._variable._data)
ds

## Lazy reading

In [3]:
ds.air

<xarray.DataArray 'air' (time: 2920, lat: 25, lon: 53)>
[3869000 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Attributes:
    GRIB_id:       11
    GRIB_name:     TMP
    actual_range:  [185.16000366210938, 322.1000061035156]
    dataset:       NMC Reanalysis
    level_desc:    Surface
    long_name:     4xDaily Air temperature at sigma level 995
    parent_stat:   Other
    precision:     2
    statistic:     Individual Obs
    units:         degK
    var_desc:      Air temperature

## Data load for repr

In [4]:
ds["air"].isel(time=0, lat=10)

<xarray.DataArray 'air' (lon: 53)>
array([277.29   , 277.4    , 277.79   , 278.6    , 279.5    , 280.1    ,
       280.6    , 280.9    , 280.79   , 280.69998, 280.79   , 281.     ,
       280.29   , 277.69998, 273.5    , 269.     , 265.5    , 264.     ,
       265.19998, 268.1    , 269.79   , 267.9    , 263.     , 258.1    ,
       254.59999, 251.79999, 249.59999, 249.89   , 252.29999, 254.     ,
       254.29999, 255.89   , 260.     , 263.     , 261.5    , 257.29   ,
       255.5    , 258.29   , 264.     , 268.69998, 270.5    , 270.6    ,
       271.19998, 272.9    , 274.79   , 276.4    , 278.19998, 280.5    ,
       282.9    , 284.69998, 286.1    , 286.9    , 286.6    ], dtype=float32)
Coordinates:
    lat      float32 50.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
    time     datetime64[ns] 2013-01-01
Attributes:
    GRIB_id:       11
    GRIB_name:     TMP
    actual_range:  [185.16000366210938, 322.1000061035156]
    dataset:       NMC Reanalysis
    level_desc:    Surface
    long_name:     4xDaily Air temperature at sigma level 995
    parent_stat:   Other
    precision:     2
    statistic:     Individual Obs
    units:         degK
    var_desc:      Air temperature

## CuPy array on load

In [8]:
type(ds["air"].isel(time=0, lat=10).data)

cupy._core.core.ndarray

In [9]:
type(ds["air"].isel(time=0, lat=10).load().data)

cupy._core.core.ndarray

## Load to host

In [10]:
ds.air.as_numpy()

<xarray.DataArray 'air' (time: 2920, lat: 25, lon: 53)>
array([[[241.2    , 242.5    , 243.5    , ..., 232.79999, 235.5    ,
         238.59999],
        [243.79999, 244.5    , 244.7    , ..., 232.79999, 235.29999,
         239.29999],
        [250.     , 249.79999, 248.89   , ..., 233.2    , 236.39   ,
         241.7    ],
        ...,
        [296.6    , 296.19998, 296.4    , ..., 295.4    , 295.1    ,
         294.69998],
        [295.9    , 296.19998, 296.79   , ..., 295.9    , 295.9    ,
         295.19998],
        [296.29   , 296.79   , 297.1    , ..., 296.9    , 296.79   ,
         296.6    ]],

       [[242.09999, 242.7    , 243.09999, ..., 232.     , 233.59999,
         235.79999],
        [243.59999, 244.09999, 244.2    , ..., 231.     , 232.5    ,
         235.7    ],
        [253.2    , 252.89   , 252.09999, ..., 230.79999, 233.39   ,
         238.5    ],
...
        [293.69   , 293.88998, 295.38998, ..., 295.09   , 294.69   ,
         294.29   ],
        [296.29   , 297.19   , 297.59   , ..., 295.29   , 295.09   ,
         294.38998],
        [297.79   , 298.38998, 298.49   , ..., 295.69   , 295.49   ,
         295.19   ]],

       [[245.09   , 244.29   , 243.29   , ..., 241.68999, 241.48999,
         241.79   ],
        [249.89   , 249.29   , 248.39   , ..., 239.59   , 240.29   ,
         241.68999],
        [262.99   , 262.19   , 261.38998, ..., 239.89   , 242.59   ,
         246.29   ],
        ...,
        [293.79   , 293.69   , 295.09   , ..., 295.29   , 295.09   ,
         294.69   ],
        [296.09   , 296.88998, 297.19   , ..., 295.69   , 295.69   ,
         295.19   ],
        [297.69   , 298.09   , 298.09   , ..., 296.49   , 296.19   ,
         295.69   ]]], dtype=float32)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Attributes:
    GRIB_id:       11
    GRIB_name:     TMP
    actual_range:  [185.16000366210938, 322.1000061035156]
    dataset:       NMC Reanalysis
    level_desc:    Surface
    long_name:     4xDaily Air temperature at sigma level 995
    parent_stat:   Other
    precision:     2
    statistic:     Individual Obs
    units:         degK
    var_desc:      Air temperature

In [11]:
type(ds.air.as_numpy().data)

numpy.ndarray

In [12]:
type(ds.air.mean("time").load().data)

cupy._core.core.ndarray

## GroupBy with flox

Requires

1. flox main branch?
2. https://github.com/ml31415/numpy-groupies/pull/63

In [25]:
ds.air.groupby("time.month").mean(engine="numpy")

<xarray.DataArray 'air' (month: 12, lat: 25, lon: 53)>
array([[[246.34975, 246.38591, 246.21495, ..., 243.06096, 244.08774,
         245.64653],
        [248.8575 , 248.90729, 248.71028, ..., 241.52846, 243.50845,
         246.7545 ],
        [251.57713, 251.19649, 250.71451, ..., 243.39871, 246.78438,
         251.56555],
        ...,
        [295.8504 , 295.24423, 295.2271 , ..., 295.1864 , 294.65726,
         294.04868],
        [296.54468, 296.47   , 296.16025, ..., 295.35614, 295.0814 ,
         294.53015],
        [297.15448, 297.23843, 297.04913, ..., 296.01816, 295.7758 ,
         295.63678]],

       [[246.6771 , 246.40562, 245.94829, ..., 241.85826, 243.00186,
         244.44365],
        [247.79994, 247.75986, 247.4774 , ..., 240.64687, 242.26611,
         245.06642],
        [249.07076, 248.57227, 247.94246, ..., 242.42851, 245.33334,
         249.72244],
...
        [297.8426 , 297.14062, 296.98776, ..., 297.96893, 297.56882,
         297.16125],
        [298.58783, 298.4203 , 297.96912, ..., 298.16418, 298.194  ,
         297.90833],
        [298.81146, 298.8566 , 298.6211 , ..., 298.7296 , 298.7519 ,
         298.81894]],

       [[247.97087, 248.02097, 247.91281, ..., 239.77168, 241.02357,
         242.62805],
        [249.73338, 250.16028, 250.48564, ..., 238.78947, 240.96451,
         244.11601],
        [252.02939, 251.53108, 251.36617, ..., 238.07532, 241.91273,
         247.06967],
        ...,
        [296.76517, 295.97668, 295.88925, ..., 296.456  , 296.09137,
         295.65768],
        [297.46817, 297.38034, 297.0443 , ..., 296.85565, 296.84668,
         296.52142],
        [297.88092, 297.98676, 297.7755 , ..., 297.60034, 297.5654 ,
         297.53772]]], dtype=float32)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12